In [1]:
CACHE_DIR = "../cache"

In [42]:
from transformers import VideoMAEConfig, VideoMAEModel, VideoMAEFeatureExtractor, AutoImageProcessor, VideoMAEForPreTraining, VideoMAEForVideoClassification
import numpy as np
import torch

In [47]:
processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base", cache_dir=CACHE_DIR)
videomae = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base", cache_dir=CACHE_DIR)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [46]:
num_frames = 16
video = list(np.random.randint(0, 256, (num_frames, 3, 1024, 512)))

model_input = processor([video, video], return_tensors="pt", padding=True)
pixel_values = model_input.pixel_values
print(pixel_values.shape)

torch.Size([2, 16, 3, 224, 224])


In [52]:
outputs = videomae(pixel_values)

sequence_output = outputs[0]
batch_size, seq_len, hidden_size = sequence_output.shape
print(sequence_output.shape)

torch.Size([2, 1568, 768])


In [53]:
video_sequence_output = sequence_output.reshape(batch_size, num_frames, -1, hidden_size)
print(video_sequence_output.shape)

torch.Size([2, 16, 98, 768])


In [56]:
# torch.Size([2, 16, 98, 768]) to torch.Size([2, 16, 768]) by compute mean
video_mean_output = torch.mean(video_sequence_output, dim=2)
print(video_mean_output.shape)

torch.Size([2, 16, 768])


In [49]:
pretrain_video_mae = VideoMAEForPreTraining.from_pretrained("MCG-NJU/videomae-base", cache_dir=CACHE_DIR)
pretrain_video_mae

VideoMAEForPreTraining(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x VideoMAELayer(
          (attention): VideoMAEAttention(
            (attention): VideoMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (dense): Linear(in_f